# DeepRoof-2026: Result Visualization

This notebook demonstrates how to load and visualize the results from the inference pipeline.

In [ ]:
import rasterio
from rasterio.plot import show
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable

%matplotlib inline

In [ ]:
# Paths (Update these to your specific files)
IMAGE_PATH = '../data/test_image.tif' # Example path
GEOJSON_PATH = '../results/output.geojson' # Example output path

In [ ]:
# Load Satellite Image
try:
    src = rasterio.open(IMAGE_PATH)
    print(f"Loaded image: {src.meta['width']}x{src.meta['height']}, CRS: {src.crs}")
except Exception as e:
    print(f"Could not load image at {IMAGE_PATH}: {e}")
    src = None

In [ ]:
# Load Predictions
try:
    gdf = gpd.read_file(GEOJSON_PATH)
    print(f"Loaded {len(gdf)} polygons.")
    print(gdf.head())
except Exception as e:
    print(f"Could not load GeoJSON at {GEOJSON_PATH}: {e}")
    gdf = None

In [ ]:
# Visualization
if src and gdf is not None:
    fig, ax = plt.subplots(figsize=(15, 15))
    
    # Plot Image
    show(src, ax=ax)
    
    # Plot Polygons Color-Coded by Slope
    # Normalize slope from 0 to 60 degrees (common roof range)
    norm = Normalize(vmin=0, vmax=60)
    cmap = 'jet'
    
    # Filter valid slopes (ignore -1 if any)
    valid_gdf = gdf[gdf['slope'] >= 0]
    
    if not valid_gdf.empty:
        valid_gdf.plot(column='slope', ax=ax, cmap=cmap, alpha=0.6, 
                       legend=False, norm=norm, edgecolor='white', linewidth=0.5)
        
        # Add Colorbar
        cbar = plt.colorbar(ScalarMappable(norm=norm, cmap=cmap), ax=ax, fraction=0.035, pad=0.04)
        cbar.set_label('Roof Slope (Degrees)')
    else:
        # Just plot geometries if no valid attribute
        gdf.plot(ax=ax, facecolor='red', alpha=0.5, edgecolor='white')

    ax.set_title("DeepRoof-2026: Roof Extraction & Geometry Estimation")
    plt.show()
else:
    print("Skipping visualization due to missing data.")